# Baseline experiments

Импорт либ

In [1]:
# !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu126

In [3]:
import gdown
import zipfile
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
import cv2 as cv
import seaborn as sns
from itertools import chain
from sklearn.preprocessing import label_binarize
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple
import random
import torch
from torchvision.transforms import ToTensor
from urllib.request import urlopen
import timm
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from sklearn.svm import SVC
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

### EfficientNetV2-S (input 450x450)

In [42]:
imagefolder = "C:/Users/User/images/450/"

Feature Extraction

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_s.in21k_ft_in1k',
    pretrained=True,
    features_only=True,
)

model = model.to(device)
model.eval()
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    with torch.no_grad():
        outputs = model(images)
        last_feature_map = outputs[-1]  # Get the last feature map
        # Flatten the feature map
        features = last_feature_map.view(last_feature_map.size(0), -1)
        return features

# Создаём DataLoader для последующего Feature extraction'a
dataset = ImageFolder(imagefolder, transform=transforms)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)

num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_size = round(train_test_ratio*len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [05:10<00:00,  3.35it/s]

Train size is 26586
torch.Size([26586, 25600]) torch.Size([6646, 25600])
torch.Size([26586]) torch.Size([6646])


In [6]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_S_450.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_S_450.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_S_450.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_S_450.csv", y_test_numpy, delimiter=",")

In [4]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V2_S_450.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_S_450.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_S_450.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_S_450.csv", delimiter=",")

In [5]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 1280)
(6646, 1280)
(26586,)
(6646,)


In [7]:
# Здесь эскпериментируем с модельками
X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(
    X_train_numpy, y_train_numpy, train_size=0.75)

max_depth_list = [3, 5, 7, 9]
min_child_weight_list = [1, 3, 5]
gamma_list = [i/10 for i in range(5)]
subsample_list = [i/10 for i in range(10)]
colsample_bytree_list = [i/10 for i in range(10)]
reg_alpha_list = [1e-5, 1e-2, 0.1, 1, 100]

best_params = {
    'max_depth': None,
    'min_child_weight': None,
    'gamma': None,
    'subsample': None,
    'colsample_bytree': None,
    'reg_alpha': None
}

print('Learning max_depth and min_child_weight:')

best_accuracy = 0
for max_depth in max_depth_list:
    for min_child_weight in min_child_weight_list:
            xgb = XGBClassifier(
                tree_method='gpu_hist', 
                gpu_id=0, 
                max_depth=max_depth, 
                min_child_weight=min_child_weight
            )
            xgb.fit(X_train_numpy, y_train_numpy)
            y_pred = xgb.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['max_depth'] = max_depth
                best_params['min_child_weight'] = min_child_weight
            print(f"Accuracy: {accuracy}")
            
print('Learning gamma:')

best_accuracy = 0
for gamma in gamma_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=gamma
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['gamma'] = gamma
    print(f"Accuracy: {accuracy}")
    
print('Learning subsample and colsample_bytree:')

best_accuracy = 0
for subsample in subsample_list:
    for colsample_bytree in colsample_bytree_list:
        xgb = XGBClassifier(
            tree_method='gpu_hist', 
            gpu_id=0, 
            max_depth=best_params['max_depth'], 
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=subsample,
            colsample_bytree=colsample_bytree
        )
        xgb.fit(X_train_numpy, y_train_numpy)
        y_pred = xgb.predict(X_val_numpy)
        accuracy = np.mean([y_pred == y_val_numpy])
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params['subsample'] = subsample
            best_params['colsample_bytree'] = colsample_bytree
        print(f"Accuracy: {accuracy}")

print('Learning reg_alpha:')

best_accuracy = 0
for reg_alpha in reg_alpha_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=best_params['gamma'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=reg_alpha
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['reg_alpha'] = reg_alpha
    print(f"Accuracy: {accuracy}")

Learning max_depth and min_child_weight:
Accuracy: 0.8259365127125019
Accuracy: 0.8277418384233489
Accuracy: 0.8269896193771626
Accuracy: 0.8730254250037611
Accuracy: 0.8722732059575748
Accuracy: 0.8733263126222356
Accuracy: 0.8870166992628253
Accuracy: 0.8835564916503685
Accuracy: 0.8852113735519783
Accuracy: 0.8834060478411313
Accuracy: 0.8847600421242666
Accuracy: 0.8843087106965548
Learning gamma:
Accuracy: 0.8870166992628253
Accuracy: 0.8816007221302843
Accuracy: 0.8716714307206258
Accuracy: 0.868812998345118
Accuracy: 0.8639987964495262
Learning subsample and colsample_bytree:
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.05837219798405296
Accuracy: 0.5251993380472394
Accuracy: 0.7562810290356552
Accuracy: 0.7767413870919212
Accuracy: 0.78275913946

In [9]:
print('Reducing learning rate and increasing n_estimators:')

X_train_numpy = np.genfromtxt("X_train_V2_S_450.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_S_450.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_S_450.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_S_450.csv", delimiter=",")

xgb = XGBClassifier(
    tree_method='gpu_hist', 
    gpu_id=0, 
    max_depth=best_params['max_depth'], 
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    learning_rate=0.05,
    n_estimators=5000
)

xgb.fit(X_train_numpy, y_train_numpy)
y_pred = xgb.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print(f"Accuracy: {accuracy}")
    
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {best_params}')

Reducing learning rate and increasing n_estimators:
Accuracy: 0.9437255492025278
Best model accuracy: 0.9437255492025278
Best model params: {'max_depth': 7, 'min_child_weight': 1, 'gamma': 0.0, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_alpha': 1e-05}


## Обучение на фичах других EfficientNet'ов

### EfficientNet B5

In [12]:
imagefolder = "C:/Users/User/images/456_white/"

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'efficientnet_b5.sw_in12k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 2048])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [07:10<00:00,  2.41it/s]

Train size is 26586
torch.Size([26586, 2048]) torch.Size([6646, 2048])
torch.Size([26586]) torch.Size([6646])


In [14]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V1_B5_456.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V1_B5_456.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V1_B5_456.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V1_B5_456.csv", y_test_numpy, delimiter=",")

In [13]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V1_B5_456.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V1_B5_456.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V1_B5_456.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V1_B5_456.csv", delimiter=",")

In [14]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 2048)
(6646, 2048)
(26586,)
(6646,)


In [15]:
# Здесь эскпериментируем с модельками
X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(
    X_train_numpy, y_train_numpy, train_size=0.75)

max_depth_list = [3, 5, 7, 9]
min_child_weight_list = [1, 3, 5]
gamma_list = [i/10 for i in range(5)]
subsample_list = [i/10 for i in range(10)]
colsample_bytree_list = [i/10 for i in range(10)]
reg_alpha_list = [1e-5, 1e-2, 0.1, 1, 100]

best_params = {
    'max_depth': None,
    'min_child_weight': None,
    'gamma': None,
    'subsample': None,
    'colsample_bytree': None,
    'reg_alpha': None
}

print('Learning max_depth and min_child_weight:')

best_accuracy = 0
for max_depth in max_depth_list:
    for min_child_weight in min_child_weight_list:
            xgb = XGBClassifier(
                tree_method='gpu_hist', 
                gpu_id=0, 
                max_depth=max_depth, 
                min_child_weight=min_child_weight
            )
            xgb.fit(X_train_numpy, y_train_numpy)
            y_pred = xgb.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['max_depth'] = max_depth
                best_params['min_child_weight'] = min_child_weight
            print(f"Accuracy: {accuracy}")
            
print('Learning gamma:')

best_accuracy = 0
for gamma in gamma_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=gamma
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['gamma'] = gamma
    print(f"Accuracy: {accuracy}")
    
print('Learning subsample and colsample_bytree:')

best_accuracy = 0
for subsample in subsample_list:
    for colsample_bytree in colsample_bytree_list:
        xgb = XGBClassifier(
            tree_method='gpu_hist', 
            gpu_id=0, 
            max_depth=best_params['max_depth'], 
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=subsample,
            colsample_bytree=colsample_bytree
        )
        xgb.fit(X_train_numpy, y_train_numpy)
        y_pred = xgb.predict(X_val_numpy)
        accuracy = np.mean([y_pred == y_val_numpy])
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params['subsample'] = subsample
            best_params['colsample_bytree'] = colsample_bytree
        print(f"Accuracy: {accuracy}")

print('Learning reg_alpha:')

best_accuracy = 0
for reg_alpha in reg_alpha_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=best_params['gamma'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=reg_alpha
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['reg_alpha'] = reg_alpha
    print(f"Accuracy: {accuracy}")

Learning max_depth and min_child_weight:
Accuracy: 0.8354144726944486
Accuracy: 0.8390251241161426
Accuracy: 0.8364675793591094
Accuracy: 0.8871671430720626
Accuracy: 0.8840078230780803
Accuracy: 0.8859635925981646
Accuracy: 0.9017601925680758
Accuracy: 0.8949902211523996
Accuracy: 0.8976982097186701
Accuracy: 0.8993530916202799
Accuracy: 0.8961937716262975
Accuracy: 0.8963442154355348
Learning gamma:
Accuracy: 0.9017601925680758
Accuracy: 0.8919813449676546
Accuracy: 0.8885211373551979
Accuracy: 0.8832556040318941
Accuracy: 0.8831051602226568
Learning subsample and colsample_bytree:
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.05927486083947645
Accuracy: 0.5425003761095231
Accuracy: 0.7932902061080187
Accuracy: 0.7993079584775087
Accuracy: 0.8048743794

In [16]:
print('Reducing learning rate and increasing n_estimators:')

X_train_numpy = np.genfromtxt("X_train_V1_B5_456.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V1_B5_456.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V1_B5_456.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V1_B5_456.csv", delimiter=",")

xgb = XGBClassifier(
    tree_method='gpu_hist', 
    gpu_id=0, 
    max_depth=best_params['max_depth'], 
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    learning_rate=0.05,
    n_estimators=5000
)

xgb.fit(X_train_numpy, y_train_numpy)
y_pred = xgb.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print(f"Accuracy: {accuracy}")
    
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {best_params}')

Reducing learning rate and increasing n_estimators:
Accuracy: 0.9357508275654529
Best model accuracy: 0.9357508275654529
Best model params: {'max_depth': 7, 'min_child_weight': 1, 'gamma': 0.0, 'subsample': 0.8, 'colsample_bytree': 0.6, 'reg_alpha': 0.01}


### EfficientNet-v2 M

In [17]:
imagefolder = "C:/Users/User/images/480_white/"

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_m.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [06:50<00:00,  2.53it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [19]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_M_480.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_M_480.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_M_480.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_M_480.csv", y_test_numpy, delimiter=",")

In [17]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V2_M_480.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_M_480.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_M_480.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_M_480.csv", delimiter=",")

In [18]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 1280)
(6646, 1280)
(26586,)
(6646,)


In [19]:
# Здесь эскпериментируем с модельками
X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(
    X_train_numpy, y_train_numpy, train_size=0.75)

max_depth_list = [3, 5, 7, 9]
min_child_weight_list = [1, 3, 5]
gamma_list = [i/10 for i in range(5)]
subsample_list = [i/10 for i in range(10)]
colsample_bytree_list = [i/10 for i in range(10)]
reg_alpha_list = [1e-5, 1e-2, 0.1, 1, 100]

best_params = {
    'max_depth': None,
    'min_child_weight': None,
    'gamma': None,
    'subsample': None,
    'colsample_bytree': None,
    'reg_alpha': None
}

print('Learning max_depth and min_child_weight:')

best_accuracy = 0
for max_depth in max_depth_list:
    for min_child_weight in min_child_weight_list:
            xgb = XGBClassifier(
                tree_method='gpu_hist', 
                gpu_id=0, 
                max_depth=max_depth, 
                min_child_weight=min_child_weight
            )
            xgb.fit(X_train_numpy, y_train_numpy)
            y_pred = xgb.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['max_depth'] = max_depth
                best_params['min_child_weight'] = min_child_weight
            print(f"Accuracy: {accuracy}")
            
print('Learning gamma:')

best_accuracy = 0
for gamma in gamma_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=gamma
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['gamma'] = gamma
    print(f"Accuracy: {accuracy}")
    
print('Learning subsample and colsample_bytree:')

best_accuracy = 0
for subsample in subsample_list:
    for colsample_bytree in colsample_bytree_list:
        xgb = XGBClassifier(
            tree_method='gpu_hist', 
            gpu_id=0, 
            max_depth=best_params['max_depth'], 
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=subsample,
            colsample_bytree=colsample_bytree
        )
        xgb.fit(X_train_numpy, y_train_numpy)
        y_pred = xgb.predict(X_val_numpy)
        accuracy = np.mean([y_pred == y_val_numpy])
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params['subsample'] = subsample
            best_params['colsample_bytree'] = colsample_bytree
        print(f"Accuracy: {accuracy}")

print('Learning reg_alpha:')

best_accuracy = 0
for reg_alpha in reg_alpha_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=best_params['gamma'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=reg_alpha
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['reg_alpha'] = reg_alpha
    print(f"Accuracy: {accuracy}")

Learning max_depth and min_child_weight:
Accuracy: 0.8345118098390252
Accuracy: 0.8375206860237702
Accuracy: 0.8385737926884309
Accuracy: 0.8823529411764706
Accuracy: 0.8900255754475703
Accuracy: 0.8879193621182488
Accuracy: 0.900105310666466
Accuracy: 0.8930344516323153
Accuracy: 0.8946893335339251
Accuracy: 0.8885211373551979
Accuracy: 0.8925831202046036
Accuracy: 0.8937866706785016
Learning gamma:
Accuracy: 0.900105310666466
Accuracy: 0.8879193621182488
Accuracy: 0.8856627049796901
Accuracy: 0.8808485030840981
Accuracy: 0.8772378516624041
Learning subsample and colsample_bytree:
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.05611554084549421
Accuracy: 0.5500225665713856
Accuracy: 0.7862193470738679
Accuracy: 0.7812547013690386
Accuracy: 0.792387543252

In [20]:
print('Reducing learning rate and increasing n_estimators:')

X_train_numpy = np.genfromtxt("X_train_V2_M_480.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_M_480.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_M_480.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_M_480.csv", delimiter=",")

xgb = XGBClassifier(
    tree_method='gpu_hist', 
    gpu_id=0, 
    max_depth=best_params['max_depth'], 
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    learning_rate=0.05,
    n_estimators=5000
)

xgb.fit(X_train_numpy, y_train_numpy)
y_pred = xgb.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print(f"Accuracy: {accuracy}")
    
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {best_params}')

Reducing learning rate and increasing n_estimators:
Accuracy: 0.9384592235931387
Best model accuracy: 0.9384592235931387
Best model params: {'max_depth': 7, 'min_child_weight': 1, 'gamma': 0.0, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_alpha': 0.1}


### EfficientNet-v2 L

In [24]:
imagefolder = "C:/Users/User/images/480_white/"

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_l.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [08:34<00:00,  2.02it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [26]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_L_480.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_L_480.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_L_480.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_L_480.csv", y_test_numpy, delimiter=",")

In [21]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V2_L_480.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_L_480.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_L_480.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_L_480.csv", delimiter=",")

In [22]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 1280)
(6646, 1280)
(26586,)
(6646,)


In [23]:
# Здесь эскпериментируем с модельками
X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(
    X_train_numpy, y_train_numpy, train_size=0.75)

max_depth_list = [3, 5, 7, 9]
min_child_weight_list = [1, 3, 5]
gamma_list = [i/10 for i in range(5)]
subsample_list = [i/10 for i in range(10)]
colsample_bytree_list = [i/10 for i in range(10)]
reg_alpha_list = [1e-5, 1e-2, 0.1, 1, 100]

best_params = {
    'max_depth': None,
    'min_child_weight': None,
    'gamma': None,
    'subsample': None,
    'colsample_bytree': None,
    'reg_alpha': None
}

print('Learning max_depth and min_child_weight:')

best_accuracy = 0
for max_depth in max_depth_list:
    for min_child_weight in min_child_weight_list:
            xgb = XGBClassifier(
                tree_method='gpu_hist', 
                gpu_id=0, 
                max_depth=max_depth, 
                min_child_weight=min_child_weight
            )
            xgb.fit(X_train_numpy, y_train_numpy)
            y_pred = xgb.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['max_depth'] = max_depth
                best_params['min_child_weight'] = min_child_weight
            print(f"Accuracy: {accuracy}")
            
print('Learning gamma:')

best_accuracy = 0
for gamma in gamma_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=gamma
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['gamma'] = gamma
    print(f"Accuracy: {accuracy}")
    
print('Learning subsample and colsample_bytree:')

best_accuracy = 0
for subsample in subsample_list:
    for colsample_bytree in colsample_bytree_list:
        xgb = XGBClassifier(
            tree_method='gpu_hist', 
            gpu_id=0, 
            max_depth=best_params['max_depth'], 
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=subsample,
            colsample_bytree=colsample_bytree
        )
        xgb.fit(X_train_numpy, y_train_numpy)
        y_pred = xgb.predict(X_val_numpy)
        accuracy = np.mean([y_pred == y_val_numpy])
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params['subsample'] = subsample
            best_params['colsample_bytree'] = colsample_bytree
        print(f"Accuracy: {accuracy}")

print('Learning reg_alpha:')

best_accuracy = 0
for reg_alpha in reg_alpha_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=best_params['gamma'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=reg_alpha
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['reg_alpha'] = reg_alpha
    print(f"Accuracy: {accuracy}")

Learning max_depth and min_child_weight:
Accuracy: 0.8325560403189409
Accuracy: 0.8349631412667369
Accuracy: 0.8357153603129232
Accuracy: 0.8903264630660448
Accuracy: 0.8858131487889274
Accuracy: 0.8864149240258763
Accuracy: 0.8975477659094329
Accuracy: 0.8979990973371446
Accuracy: 0.8957424401985858
Accuracy: 0.8889724687829096
Accuracy: 0.8951406649616368
Accuracy: 0.8961937716262975
Learning gamma:
Accuracy: 0.8979990973371446
Accuracy: 0.8958928840078231
Accuracy: 0.8868662554535881
Accuracy: 0.8825033849857078
Accuracy: 0.8779900707085904
Learning subsample and colsample_bytree:
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.0615315179780352
Accuracy: 0.5393410561155408
Accuracy: 0.7821573642244621
Accuracy: 0.7838122461260719
Accuracy: 0.7947946442003911
Acc

In [24]:
print('Reducing learning rate and increasing n_estimators:')

X_train_numpy = np.genfromtxt("X_train_V2_L_480.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_L_480.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_L_480.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_L_480.csv", delimiter=",")

xgb = XGBClassifier(
    tree_method='gpu_hist', 
    gpu_id=0, 
    max_depth=best_params['max_depth'], 
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    learning_rate=0.05,
    n_estimators=5000
)

xgb.fit(X_train_numpy, y_train_numpy)
y_pred = xgb.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print(f"Accuracy: {accuracy}")
    
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {best_params}')

Reducing learning rate and increasing n_estimators:
Accuracy: 0.9468853445681613
Best model accuracy: 0.9468853445681613
Best model params: {'max_depth': 7, 'min_child_weight': 3, 'gamma': 0.0, 'subsample': 0.9, 'colsample_bytree': 0.8, 'reg_alpha': 1e-05}


### EfficientNet-V2 S (384x384)

In [37]:
imagefolder = "C:/Users/User/images/384/"

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_s.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [04:26<00:00,  3.90it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [39]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_S_384.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_S_384.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_S_384.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_S_384.csv", y_test_numpy, delimiter=",")

In [25]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V2_S_384.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_S_384.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_S_384.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_S_384.csv", delimiter=",")

In [26]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 1280)
(6646, 1280)
(26586,)
(6646,)


In [27]:
# Здесь эскпериментируем с модельками
X_train_numpy, X_val_numpy, y_train_numpy, y_val_numpy = train_test_split(
    X_train_numpy, y_train_numpy, train_size=0.75)

max_depth_list = [3, 5, 7, 9]
min_child_weight_list = [1, 3, 5]
gamma_list = [i/10 for i in range(5)]
subsample_list = [i/10 for i in range(10)]
colsample_bytree_list = [i/10 for i in range(10)]
reg_alpha_list = [1e-5, 1e-2, 0.1, 1, 100]

best_params = {
    'max_depth': None,
    'min_child_weight': None,
    'gamma': None,
    'subsample': None,
    'colsample_bytree': None,
    'reg_alpha': None
}

print('Learning max_depth and min_child_weight:')

best_accuracy = 0
for max_depth in max_depth_list:
    for min_child_weight in min_child_weight_list:
            xgb = XGBClassifier(
                tree_method='gpu_hist', 
                gpu_id=0, 
                max_depth=max_depth, 
                min_child_weight=min_child_weight
            )
            xgb.fit(X_train_numpy, y_train_numpy)
            y_pred = xgb.predict(X_val_numpy)
            accuracy = np.mean([y_pred == y_val_numpy])
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['max_depth'] = max_depth
                best_params['min_child_weight'] = min_child_weight
            print(f"Accuracy: {accuracy}")
            
print('Learning gamma:')

best_accuracy = 0
for gamma in gamma_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=gamma
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['gamma'] = gamma
    print(f"Accuracy: {accuracy}")
    
print('Learning subsample and colsample_bytree:')

best_accuracy = 0
for subsample in subsample_list:
    for colsample_bytree in colsample_bytree_list:
        xgb = XGBClassifier(
            tree_method='gpu_hist', 
            gpu_id=0, 
            max_depth=best_params['max_depth'], 
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=subsample,
            colsample_bytree=colsample_bytree
        )
        xgb.fit(X_train_numpy, y_train_numpy)
        y_pred = xgb.predict(X_val_numpy)
        accuracy = np.mean([y_pred == y_val_numpy])
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params['subsample'] = subsample
            best_params['colsample_bytree'] = colsample_bytree
        print(f"Accuracy: {accuracy}")

print('Learning reg_alpha:')

best_accuracy = 0
for reg_alpha in reg_alpha_list:
    xgb = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0, 
        max_depth=best_params['max_depth'], 
        min_child_weight=best_params['min_child_weight'],
        gamma=best_params['gamma'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        reg_alpha=reg_alpha
    )
    xgb.fit(X_train_numpy, y_train_numpy)
    y_pred = xgb.predict(X_val_numpy)
    accuracy = np.mean([y_pred == y_val_numpy])
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params['reg_alpha'] = reg_alpha
    print(f"Accuracy: {accuracy}")

Learning max_depth and min_child_weight:
Accuracy: 0.8315029336542801
Accuracy: 0.8302993831803821
Accuracy: 0.8292462765157214
Accuracy: 0.8719723183391004
Accuracy: 0.8746803069053708
Accuracy: 0.8682112231081691
Accuracy: 0.8841582668873176
Accuracy: 0.8812998345118098
Accuracy: 0.8826538287949451
Accuracy: 0.8788927335640139
Accuracy: 0.8840078230780803
Accuracy: 0.881450278321047
Learning gamma:
Accuracy: 0.8841582668873176
Accuracy: 0.8808485030840981
Accuracy: 0.8745298630961336
Accuracy: 0.8683616669174063
Accuracy: 0.8626448021663908
Learning subsample and colsample_bytree:
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.058522641793290205
Accuracy: 0.5393410561155408
Accuracy: 0.7665112080637881
Accuracy: 0.7756882804272605
Accuracy: 0.7

In [28]:
print('Reducing learning rate and increasing n_estimators:')

X_train_numpy = np.genfromtxt("X_train_V2_S_384.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_S_384.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_S_384.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_S_384.csv", delimiter=",")

xgb = XGBClassifier(
    tree_method='gpu_hist', 
    gpu_id=0, 
    max_depth=best_params['max_depth'], 
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    learning_rate=0.05,
    n_estimators=5000
)

xgb.fit(X_train_numpy, y_train_numpy)
y_pred = xgb.predict(X_test_numpy)
accuracy = np.mean([y_pred == y_test_numpy])
print(f"Accuracy: {accuracy}")
    
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {best_params}')

Reducing learning rate and increasing n_estimators:
Accuracy: 0.9313873006319591
Best model accuracy: 0.9313873006319591
Best model params: {'max_depth': 7, 'min_child_weight': 1, 'gamma': 0.0, 'subsample': 0.9, 'colsample_bytree': 0.7, 'reg_alpha': 0.01}
